In [86]:
import pandas as pd

# Load Files
Acquisitions = pd.read_csv("./data/acquisitions.csv")
Degrees = pd.read_csv("./data/degrees.csv")
Ipos = pd.read_csv("./data/ipos.csv")
Investments = pd.read_csv("./data/investments.csv")

print("="*60)
print("Dataset 로드 완료!")
print("="*60)

Dataset 로드 완료!


## Acquisitions 데이터 전처리

In [5]:
# 컬럼 제거
Acquisitions_copy = Acquisitions.copy()
Acquisitions_copy = Acquisitions_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [9]:
# acquiring_object_id, acquired_object_id 컬럼명 변경
alter_columns = Acquisitions_copy.rename(columns={
    'acquiring_object_id': 'acquiring_c_id',
    'acquired_object_id': 'acquired_c_id'
})

In [94]:
# acquired_at 형 변환
Acquisitions_copy['acquired_at'] = pd.to_datetime(Acquisitions_copy['acquired_at'])

# acquired_at 결측값 플래그 컬럼 생성
Acquisitions_copy["is_acquisitions_acq_at_missing"] = Acquisitions_copy["acquired_at"].isna().astype(int)

In [95]:
Acquisitions_copy

,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,is_acquisitions_acq_at_missing
0,1,c:11,c:10,NaN,2.000000e+07,USD,2007-05-30,0
1,7,c:59,c:72,cash,6.000000e+07,USD,2007-07-01,0
2,8,c:24,c:132,cash,2.800000e+08,USD,2007-05-01,0
3,9,c:59,c:155,cash,1.000000e+08,USD,2007-06-01,0
4,10,c:212,c:215,cash,2.500000e+07,USD,2007-07-01,0
...,...,...,...,...,...,...,...,...
9557,10525,c:267859,c:285995,stock,2.220000e+08,USD,2013-12-11,0
9558,10526,c:285493,c:286006,NaN,0.000000e+00,USD,2013-12-09,0
9559,10527,c:223792,c:220208,NaN,2.600000e+10,USD,2007-07-03,0
9560,10528,c:227620,c:286176,cash,5.240000e+07,USD,2013-12-12,0


## Degrees 데이터 전처리

In [12]:
# 컬럼 제거
Degrees_copy = Degrees.copy()
Degrees_copy = Degrees_copy.drop(columns=['id', 'graduated_at', 'created_at', 'updated_at'])

In [13]:
# object_id 컬럼명 변경
alter_columns = Degrees_copy.rename(columns={
    'object_id': 'degrees_p_id'
})

In [82]:
# degree_type 카테고리화
import pandas as pd
import re

def classify_degree(x):
    s = str(x).lower().strip()

    # -------------------------
    # 1) 전문 (associate, 고졸 포함)
    # -------------------------
    if (
        "associate" in s
        or "community college" in s
        or "high school" in s
        or "ged" in s
        or re.search(r"\bhs\b", s)
        or "as" in s
        or "aa" in s
        or s in ["aas", "a.a.s.", "a.s.", "a.a.", "ad", "diploma", "certificate", "vocational", "technical", "tech school", "trade school", "2-year degree", "two-year degree", "sub-degree", "higher diploma", "hnd"]
    ):
        return "전문"

    # -------------------------
    # 2) 학사 (BA/BS/BSc, bachelor 등)
    # -------------------------
    if (
        "bachelor" in s
        or "undergrad" in s
        or re.search(r"\bb\.?a\.?\b", s)   # BA, B.A.
        or re.search(r"\bb\.?s\.?\b", s)   # BS, B.S.
        or "bsc" in s
        or "bcs" in s 
        or "bfa" in s
        or "be" in s
        or "llb" in s
        or "basc" in s
        or "sc.b" in s
        or "sc. b." in s
        or "sc b" in s
        or "s.b" in s
        or "bba" in s
        or "s.b." in s
        or "btech" in s
        or "bba" in s
        or "bcom" in s
        or "scb" in s
        or "sb" in s
        or "bba" in s 
        or "bbs" in s
        or "bid" in s
        or s in ["beng", "b.eng", "bengineering", "baur", "barch", "bva", "bm", "bmus", "bn", "bsn", "bph", "bba", "bcomm", "blaw", "bsw", "b.ed", "bed", "beng(hons)", "bth", "bpharm", "bsee", "bsce", "bsme", "bmet", "bis", "bachlor", "bse"]
    ):
        return "학사"

    # -------------------------
    # 3) 석사 (master, MS, MSc, MBA 등)
    # -------------------------
    if (
        "master" in s
        or "graduate school" in s
        or "grad school" in s
        or "mba" in s
        or re.search(r"\bm\.?s\.?\b", s)   # MS, M.S.
        or "msc" in s
        or "ma" in s
        or "miage" in s
        or "msba" in s
        or "mphil" in s
        or "llm" in s
        or "mpa" in s
        or "med" in s
        or "mse" in s
        or "meng" in s
        or "eem" in s
        or "mea" in s
        or s in ["mfa", "mim", "mlis", "mm", "mn", "msn", "march", "m.div.", "mha", "mpp", "msw", "macc", "mfin", "m.ed.", "mcm", "mlt", "a.m.", "m.a", "m.a.", "pg"]
    ):
        return "석사"

    # -------------------------
    # 4) 박사 (PhD, Doctor, MD, JD, EdD, DPhil 등)
    # -------------------------
    if (
        "phd" in s
        or "doctorate" in s
        or "doctoral" in s
        or "doctor of" in s
        or s in ["md", "jd", "edd", "dphil"]
        or "ds" in s
        or "ph d" in s 
        or "ph.d" in s
        or s in ["dba", "psy.d", "psyd", "dpt", "dvm", "aud", "dc", "do", "drph", "scd", "thd", "j.s.d", "s.j.d", "ll.d", "abd", "j.d", "j d", "jd.", "j.d."]
    ):
        return "박사"

    # -------------------------
    # 5) 그 외는 모두 Others
    # -------------------------
    return "Others"

# 새 컬럼 생성
Degrees_copy["cat_degrees_degree_type"] = Degrees_copy["degree_type"].apply(classify_degree)

# 2) categorical로 변환
category_order = ["전문", "학사", "석사", "박사", "Others"]

Degrees_copy["cat_degrees_degree_type"] = pd.Categorical(
    Degrees_copy["cat_degrees_degree_type"],
    categories=category_order,
    ordered=True
)

In [83]:
# 매핑 딕셔너리 정의
degree_mapping = {
    "전문": 0,
    "학사": 1,
    "석사": 2,
    "박사": 3,
    "Others": 99
}

# 'cat_degrees_degree_type' 컬럼의 값을 매핑
Degrees_copy["degree_level"] = Degrees_copy["cat_degrees_degree_type"].map(degree_mapping)

# degree 결측값 플래그 컬럼 생성
Degrees_copy["is_degree_missing"] = Degrees_copy["degree_type"].isna().astype(int)


In [84]:
# subject 카테고리화

# 1) 매핑 함수 정의
def map_subject(sub):
    s = str(sub).lower()

    # Engineering & Computer Science
    if any(k in s for k in ["computer", "software", "engineering", "information", "data", "robot", "machine", "ai", "electrical"]):
        return "Engineering & Computer Science"
    
    # Natural Science
    if any(k in s for k in ["physics", "chemistry", "biology", "math", "earth", "natural"]):
        return "Natural Science"
    
    # Biotechnology / Medical / Health Science
    if any(k in s for k in ["medicine", "medical", "bio", "neuro", "pharma", "health"]):
        return "Biotechnology / Medical / Health Science"
    
    # Business / Management / Economics
    if any(k in s for k in ["business", "mba", "management", "finance", "economics", "marketing", "account"]):
        return "Business / Management / Economics"
    
    # Social Science
    if any(k in s for k in ["psychology", "sociology", "anthropology", "political", "politics"]):
        return "Social Science"
    
    # Humanities / Arts
    if any(k in s for k in [
    "philosophy", "history", "art", "literature", "humanities",
    "linguistics", "linguistic", "language", 
    "english", "korean", "french", "german", "chinese", "japanese", "spanish",
    "translation", "interpretation", "literary"
    ]):
        return "Humanities / Arts"

    # Law / Public Policy
    if any(k in s for k in ["law", "legal", "policy", "public administration"]):
        return "Law / Public Policy"
    
    # Education
    if any(k in s for k in ["education", "teaching", "pedagogy"]):
        return "Education"
    
    # Design / Architecture
    if any(k in s for k in ["design", "architecture", "architect"]):
        return "Design / Architecture"
    
    return "Others / Unknown"

# 2) 카테고리 매핑 적용
Degrees_copy["cat_degrees_subject"] = Degrees_copy["subject"].apply(map_subject)

# 3) 카테고리 순서 정의
category_order = [
    "Engineering & Computer Science",
    "Natural Science",
    "Biotechnology / Medical / Health Science",
    "Business / Management / Economics",
    "Social Science",
    "Humanities / Arts",
    "Law / Public Policy",
    "Education",
    "Design / Architecture",
    "Others / Unknown"
]

# 4) Categorical 타입으로 변환
Degrees_copy["cat_degrees_subject"] = pd.Categorical(
    Degrees_copy["cat_degrees_subject"],
    categories=category_order,
    ordered=True
)

In [85]:
Degrees_copy

,object_id,degree_type,subject,institution,cat_degrees_degree_type,degree_level,is_degree_missing,cat_degrees_subject
0,p:6117,MBA,NaN,NaN,석사,2,0,Others / Unknown
1,p:6136,BA,"English, French","Washington University, St. Louis",학사,1,0,Humanities / Arts
2,p:6136,MS,Mass Communication,Boston University,석사,2,0,Others / Unknown
3,p:6005,MS,Internet Technology,University of Greenwich,석사,2,0,Others / Unknown
4,p:5832,BCS,"Computer Science, Psychology",Rice University,학사,1,0,Engineering & Computer Science
...,...,...,...,...,...,...,...,...
109605,p:268589,CPA,NaN,American Institute of Certtified Public Accoun...,Others,99,0,Others / Unknown
109606,p:268527,MS & BS,Engineering,South China University of Technology,학사,1,0,Engineering & Computer Science
109607,p:268527,PhD,Engineering,Clarkson University,박사,3,0,Engineering & Computer Science
109608,p:268528,B.S.,Electrical Engineering,Colorado State University,학사,1,0,Engineering & Computer Science


## investment 데이터 전처리

In [87]:
# 컬럼 제거
Investments_copy = Investments.copy()
Investments_copy = Investments_copy.drop(columns=['id', 'created_at', 'updated_at'])

In [88]:
# funded_object_id, investor_object_id 컬럼명 변경
alter_columns = Investments_copy.rename(columns={
    'funded_object_id': 'invested_c_id',
    'investor_object_id': 'investor_cfp_id'
})

In [89]:
Investments_copy

,funding_round_id,funded_object_id,investor_object_id
0,1,c:4,f:1
1,1,c:4,f:2
2,3,c:5,f:4
3,4,c:5,f:1
4,4,c:5,f:5
...,...,...,...
80897,57882,c:285244,p:267312
80898,57912,c:60389,p:268570
80899,57929,c:286183,p:365
80900,57929,c:286183,p:176466


## ipos 데이터 전처리

In [90]:
# 컬럼 제거
Ipos_copy = Ipos.copy()
Ipos_copy = Ipos_copy.drop(columns=['id', 'source_url', 'source_description', 'created_at', 'updated_at'])

In [92]:
# object_id 컬럼명 변경
alter_columns = Ipos_copy.rename(columns={
    'object_id': 'ipos_c_id'
})

In [96]:
# public_at 형 변환
Ipos_copy['public_at'] = pd.to_datetime(Ipos_copy['public_at'])

# public_at 결측값 플래그 컬럼 생성
Ipos_copy["is_ipos_public_at_missing"] = Ipos_copy["public_at"].isna().astype(int)

In [99]:
# 1) stock_symbol → exchange 추출
Ipos_copy["exchange"] = Ipos_copy["stock_symbol"].str.split(":").str[0]

# 2) 거래소 티어 매핑 함수
def map_tier(exchange: str) -> str:
    if not isinstance(exchange, str) or exchange.strip() == "":
        return "Unknown"
    ex = exchange.strip().upper()

    # --- Global Tier 1 ---
    global_tier1 = {
        "NASDAQ","NAS","NASDAQGM","NASDAQGS","NASDAQCM","NASDAQ NMS","NASDAQ NMQ",
        "NYSE","NYQ","NYE","NYA",
        "HKEX","HKG","HKSE","HK","HK700",
        "TSE","TYO",        # Tokyo Stock Exchange Main
        "LSE","LON",        # LSE Main
        "KOSPI"
    }
    if ex in global_tier1:
        return "Global_Tier1"

    # --- Growth / Venture ---
    growth_venture = {
        "KOSDAQ",
        "AIM",              # LSE AIM
        "MOTHERS",          # TSE Mothers
        "CHINEXT","STAR","GEM"
    }
    if ex in growth_venture:
        return "Growth_Venture"

    # --- Regional Main ---
    regional_main = {
        "TSX","TSXV",
        "ASX",
        "SGX"
    }
    if ex in regional_main:
        return "Regional_Main"

    # --- OTC / 기타 낮은 인지도 거래소 ---
    otc_other = {
        "OTC","OTCBB","OTCQB","OTCQX","OTCM","OTCMKTS",
        "PINK","GTSM"
    }
    if ex in otc_other:
        return "OTC_Other"

    # 그 외는 Unknown
    return "Unknown"


# 3) 실제 매핑 적용
Ipos_copy["ipo_market_tier"] = Ipos_copy["exchange"].apply(map_tier)


# 4) pd.Categorical 생성
tier_order = ["Global_Tier1", "Growth_Venture", "Regional_Main", "OTC_Other", "Unknown"]

Ipos_copy["cat_ipo_stock_symbol"] = pd.Categorical(
    Ipos_copy["ipo_market_tier"],
    categories=tier_order,
    ordered=True
)

# 5) 변환 컬럼 드롭
Ipos_copy = Ipos_copy.drop(columns=['exchange', 'ipo_market_tier'])

In [100]:
Ipos_copy

,ipo_id,object_id,valuation_amount,valuation_currency_code,raised_amount,raised_currency_code,public_at,stock_symbol,is_ipos_public_at_missing,cat_ipo_stock_symbol
0,1,c:1654,0.000000e+00,USD,0.000000e+00,USD,1980-12-19,NASDAQ:AAPL,0,Global_Tier1
1,2,c:1242,0.000000e+00,USD,0.000000e+00,NaN,1986-03-13,NASDAQ:MSFT,0,Global_Tier1
2,3,c:342,0.000000e+00,USD,0.000000e+00,NaN,1969-06-09,NYSE:DIS,0,Global_Tier1
3,4,c:59,0.000000e+00,USD,0.000000e+00,NaN,2004-08-25,NASDAQ:GOOG,0,Global_Tier1
4,5,c:317,1.000000e+11,USD,0.000000e+00,NaN,1997-05-01,NASDAQ:AMZN,0,Global_Tier1
...,...,...,...,...,...,...,...,...,...,...
1254,1372,c:267859,0.000000e+00,USD,0.000000e+00,USD,NaT,NYSE:EBS,1,Global_Tier1
1255,1373,c:39330,0.000000e+00,USD,7.250000e+08,USD,2013-12-11,NYSE:ARMK,0,Global_Tier1
1256,1374,c:220208,0.000000e+00,USD,2.350000e+09,USD,2008-12-08,NYSE:HLT,0,Global_Tier1
1257,1375,c:243568,0.000000e+00,USD,0.000000e+00,USD,NaT,OTC:BCLI,1,OTC_Other
